In [2]:
import numpy as np
import pandas as pd
import os

notebook_path = os.path.abspath("")
os.chdir(notebook_path)

from nba_shot_viz import *
pd.options.mode.chained_assignment = None  # default='warn'

In [9]:
#Ingest, clean and aggregate the data
df = ingest_data(2022)
clean_df = clean_pbp_data(df)
agg_df = create_buckets(clean_df)
agg_df.head()

,PLAYER_NAME,TEAM_NAME,FGA,FGM,3PA,3PM,ABS_SCORE_DIFF_BUCKETS,RAW_MINUTES_REMAINING_BUCKETS
0,Dejounte Murray,Atlanta Hawks,1,1,0.0,0,0-5,48-45
1,Trae Young,Atlanta Hawks,1,0,1.0,0,0-5,48-45
2,Dejounte Murray,Atlanta Hawks,1,0,1.0,0,0-5,48-45
3,De'Andre Hunter,Atlanta Hawks,1,0,1.0,0,0-5,48-45
4,Dejounte Murray,Atlanta Hawks,1,0,0.0,0,0-5,48-45


In [19]:
weights = pd.read_csv('eFG_Weight.csv')
score_diff = weights['Abs Score Difference']
time_cols = weights.columns[1:-1]
weights = weights.iloc[:, 1:-1].values.flatten()
# Create a new dataframe for the transformed data
transformed_df = pd.DataFrame({'ABS_SCORE_DIFF_BUCKETS': score_diff.repeat(len(time_cols)),
                               'RAW_MINUTES_REMAINING_BUCKETS': time_cols.tolist() * len(score_diff),
                               'WEIGHT': weights})

# Reset the index
transformed_df = transformed_df.reset_index(drop=True)
transformed_df.head()

,ABS_SCORE_DIFF_BUCKETS,RAW_MINUTES_REMAINING_BUCKETS,WEIGHT
0,21+,48-45,0.5
1,21+,44-41,0.5
2,21+,40-37,0.5
3,21+,36-33,0.4
4,21+,32-29,0.4


In [31]:
#Merge Dataframes
weighted_df = agg_df.merge(transformed_df, on=['ABS_SCORE_DIFF_BUCKETS', 'RAW_MINUTES_REMAINING_BUCKETS'])
#Calculate weighted shot attempt
weighted_df['WEIGHTED_FGA'] = None 
weighted_df.loc[weighted_df['3PA'] == 0, 'WEIGHTED_FGA'] = 2 * weighted_df['WEIGHT']
weighted_df.loc[weighted_df['3PA'] != 0, 'WEIGHTED_FGA'] = 3 * weighted_df['WEIGHT']
#Calculate weighted shot make
weighted_df['WEIGHTED_FGM'] = 0
weighted_df.loc[(weighted_df['3PM'] == 1), 'WEIGHTED_FGM'] = 3 * weighted_df['WEIGHT']
weighted_df.loc[(weighted_df['3PA'] == 0) & (weighted_df['FGM'] == 1), 'WEIGHTED_FGM'] = 2 * weighted_df['WEIGHT']
weighted_df.head()

,PLAYER_NAME,TEAM_NAME,FGA,FGM,3PA,3PM,ABS_SCORE_DIFF_BUCKETS,RAW_MINUTES_REMAINING_BUCKETS,WEIGHT,WEIGHTED_FGA,WEIGHTED_FGM
0,Dejounte Murray,Atlanta Hawks,1,1,0.0,0,0-5,48-45,1.0,2.0,2.0
1,Trae Young,Atlanta Hawks,1,0,1.0,0,0-5,48-45,1.0,3.0,0.0
2,Dejounte Murray,Atlanta Hawks,1,0,1.0,0,0-5,48-45,1.0,3.0,0.0
3,De'Andre Hunter,Atlanta Hawks,1,0,1.0,0,0-5,48-45,1.0,3.0,0.0
4,Dejounte Murray,Atlanta Hawks,1,0,0.0,0,0-5,48-45,1.0,2.0,0.0


In [33]:
#Check the work 
#Show rows where either FGM or 3PM are not equal to 0
weighted_df[(weighted_df['FGM'] != 0) | (weighted_df['3PM'] != 0)].WEIGHTED_FGM.unique()

array([2.  , 3.  , 2.2 , 3.3 , 2.3 , 3.45, 2.4 , 3.6 , 1.6 , 2.4 , 4.5 ,
       2.5 , 3.75, 5.25, 3.5 , 4.  , 6.  , 5.  , 7.5 , 1.8 , 2.7 , 1.  ,
       1.5 , 0.8 , 1.2 , 0.6 , 0.9 , 1.4 , 2.1 , 2.25, 0.3 , 0.45, 0.4 ,
       0.6 ])

In [34]:
#Show rows where either FGM or 3PM are equal to 0
weighted_df[(weighted_df['FGM'] == 0) & (weighted_df['3PM'] == 0)].WEIGHTED_FGM.unique()

array([0.])

In [52]:
#Aggregate the data by player and sum
weighted_agg = weighted_df.groupby(['PLAYER_NAME']).sum().reset_index()
weighted_agg = weighted_agg[['PLAYER_NAME', 'WEIGHTED_FGA', 'WEIGHTED_FGM', 'FGA', 'FGM', '3PA', '3PM']]
#Calculate weighted eFG%
weighted_agg['WEIGHTED_eFG'] = round(weighted_agg['WEIGHTED_FGM'].astype('float') / weighted_agg['WEIGHTED_FGA'].astype('float'),3)
#Calculate eFG% by weighting 3PM as 1.5
weighted_agg['eFG'] = round((weighted_agg['FGM'] + (0.5 * weighted_agg['3PM'])) / weighted_agg['FGA'],3)
#Only keep players in the top 75 percentile of FGA
weighted_agg = weighted_agg[weighted_agg['WEIGHTED_FGA'] >= weighted_agg['WEIGHTED_FGA'].quantile(.75)]
#Calculate the difference between weighted eFG% and eFG%
weighted_agg['RAW_DIFF'] = round(weighted_agg['WEIGHTED_eFG'] - weighted_agg['eFG'],3)
weighted_agg.sort_values(by='WEIGHTED_eFG', ascending=False).head(25)

,PLAYER_NAME,WEIGHTED_FGA,WEIGHTED_FGM,FGA,FGM,3PA,3PM,WEIGHTED_eFG,eFG,RAW_DIFF
390,Nikola Jokic,2115.1,1303.85,910,580,129.0,51,0.616,0.665,-0.049
126,Domantas Sabonis,1985.25,1213.05,871,538,79.0,30,0.611,0.635,-0.024
110,Deandre Ayton,1665.5,986.60,801,478,22.0,7,0.592,0.601,-0.009
159,Giannis Antetokounmpo,2629.6,1458.15,1165,652,156.0,43,0.555,0.578,-0.023
14,Alperen Sengun,1607.35,890.50,739,419,52.0,18,0.554,0.579,-0.025
245,Joel Embiid,2683.25,1487.00,1180,657,165.0,59,0.554,0.582,-0.028
301,Kevin Durant,2008.4,1109.10,785,447,201.0,83,0.552,0.622,-0.070
143,Evan Mobley,1943.0,1063.35,872,481,90.0,19,0.547,0.562,-0.015
21,Anthony Davis,1953.3,1056.95,850,479,60.0,16,0.541,0.573,-0.032
297,Kenyon Martin Jr.,1581.95,848.60,647,370,193.0,58,0.536,0.617,-0.081
